# 🧥 Fashion MNIST: End-to-End Classifier
Short steps: load → preprocess → train → evaluate → export → app.

## 1) Setup

In [ ]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import itertools, json, os, time

print("TensorFlow:", tf.__version__)
SEED = 42
tf.keras.utils.set_random_seed(SEED)


## 2) Load & Preprocess Data

In [ ]:

# Load data
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Normalize to [0,1] and add channel dim
train_images = (train_images / 255.0).astype("float32")[..., None]
test_images  = (test_images  / 255.0).astype("float32")[..., None]

# Split validation
val_images = train_images[-6000:]
val_labels = train_labels[-6000:]
train_images = train_images[:-6000]
train_labels = train_labels[:-6000]

train_images.shape, val_images.shape, test_images.shape


### Quick EDA: sample grid

In [ ]:

# Show a 5x5 grid of training images
plt.figure(figsize=(6,6))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([]); plt.yticks([]); plt.grid(False)
    plt.imshow(train_images[i].squeeze(), cmap='gray')
    plt.xlabel(class_names[train_labels[i]])
plt.tight_layout()
plt.show()


## 3) Build Model (Simple CNN)

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


## 4) Train

In [ ]:

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_fashion_mnist.keras', monitor='val_accuracy', save_best_only=True)

history = model.fit(
    train_images, train_labels,
    epochs=15,
    batch_size=128,
    validation_data=(val_images, val_labels),
    callbacks=[early_stop, checkpoint],
    verbose=2
)

# Plot training curves (single chart per metric)
plt.figure()
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('accuracy'); plt.title('Accuracy')
plt.show()

plt.figure()
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('loss'); plt.title('Loss')
plt.show()


## 5) Evaluate

In [ ]:

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

# Confusion Matrix
y_pred = np.argmax(model.predict(test_images, verbose=0), axis=1)
cm = confusion_matrix(test_labels, y_pred)

# Plot confusion matrix
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest')
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha='right')
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(cm, class_names)

print("\nClassification report:\n")
print(classification_report(test_labels, y_pred, target_names=class_names))


## 6) Export Model for App

In [ ]:

# Save final model (Keras format and H5 for wider compatibility)
model.save("fashion_mnist_model.keras")
model.save("fashion_mnist_model.h5")

# Save class names for the app
with open("class_names.json", "w") as f:
    json.dump(class_names, f)

print("Saved: fashion_mnist_model.keras, fashion_mnist_model.h5, class_names.json")


## 7) (Optional) Convert to TensorFlow Lite

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("fashion_mnist_model.tflite", "wb") as f:
    f.write(tflite_model)
print("Saved: fashion_mnist_model.tflite")


## 8) Quick Inference Test

In [ ]:

idx = 0
sample = test_images[idx:idx+1]
probs = tf.nn.softmax(model.predict(sample, verbose=0)[0]).numpy()
pred = int(np.argmax(probs))
print("Pred:", class_names[pred], "True:", class_names[test_labels[idx]])


## 9) Streamlit App Instructions

In [ ]:

# Write helper files for the app into the project directory
import shutil, json, os

target_dir = r"/mnt/data/fashion_mnist_end_to_end"
# Copy model and class names after training
for fname in ["fashion_mnist_model.h5", "class_names.json"]:
    if os.path.exists(fname):
        shutil.copy(fname, os.path.join(target_dir, fname))
        print("Copied", fname, "->", target_dir)

print("App files:")
print("- app.py at: /mnt/data/fashion_mnist_end_to_end/app.py")
print("- requirements.txt at: /mnt/data/fashion_mnist_end_to_end/requirements.txt")

print("\nRun the app locally (terminal):")
print("  cd", target_dir)
print("  pip install -r requirements.txt")
print("  streamlit run app.py")
